# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [3]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

def query(q):
    cur.execute(q)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [6]:
# Your code here; use a subquery. No join is necessary 
query("""SELECT customerNumber,
         contactLastName,
         contactFirstName
         FROM customers
         WHERE customerNumber IN (SELECT customerNumber
                                  FROM orders
                                  WHERE orderDate = '2003-01-31')""")

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [7]:
# Your code here
query("""SELECT productName, COUNT(orderNumber) AS num_orders, 
         SUM(quantityOrdered) AS total_sold
         FROM products
         JOIN orderDetails
         USING(productCode)
         GROUP BY 1
         ORDER BY 3 DESC""")

,productName,num_orders,total_sold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [12]:
# Your code here:
# Hint: because one of the tables we'll be joining 
# has duplicate customer numbers, you should use DISTINCT

query("""SELECT productName, COUNT(DISTINCT customerName)
         FROM products
         JOIN orderdetails
         USING(productCode)
         JOIN orders
         USING(orderNumber)
         JOIN customers
         USING(customerNumber)
         GROUP BY 1
         ORDER BY 2 DESC""")

,productName,COUNT(DISTINCT customerName)
23,1940 Ford Pickup Truck,25
24,1940s Ford truck,25
25,1948 Porsche 356-A Roadster,25
26,1960 BSA Gold Star DBD34,25
27,1962 LanciaA Delta 16V,25
28,1993 Mazda RX-7,25
29,1997 BMW R 1100 S,25


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [19]:
# Your code here
query("""SELECT DISTINCT employeeNumber, firstName, lastName, o.city, o.officeCode
         FROM employees e
         JOIN offices o
         USING(officeCode)
         JOIN customers c
         ON c.salesRepEmployeeNumber = e.employeeNumber
         JOIN orders
         USING(customerNumber)
         JOIN orderDetails
         USING(orderNumber)
         WHERE productCode IN (SELECT productCode
                                  FROM products
                                  JOIN orderdetails
                                  USING(productCode)
                                  JOIN orders
                                  USING(orderNumber)
                                  GROUP BY 1
                                  HAVING COUNT(DISTINCT customerNumber) < 20)
         ORDER BY 1""")

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [26]:
# Your code here
query("""SELECT employeeNumber, firstName, lastName,
         COUNT(customerNumber) AS num_cust
         FROM employees e
         JOIN customers c
         ON e.employeeNumber = c.salesRepEmployeeNumber
         GROUP BY 1
         HAVING AVG(creditLimit) > 15000;""")

,employeeNumber,firstName,lastName,num_cust
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!